[View in Colaboratory](https://colab.research.google.com/github/lizsaccoccia/RW_blog/blob/master/Copy_of_WPSI_reformat_data.ipynb)

# **Create master dataframe for machine learning model**
```
Project: Water Peace and Security Initiative Pillar 1A
Author: Samantha Kuzma
Date: July 13, 2018
```



### Model Input Categories
1.   Socioeconomic data (soc)
>1. Infant Mortality (soc_inf)
>2.   Ethnic Fractionalization (soc_eth)
>3.   Population (soc_pop)

2.   Water data
>1. Evapotranspiration (wat_et)
>2.   Precipitation (wat_spi)
>3.  Baseline Water Stress (wat_bws)

3.   Governance/ Institutional Capacity
>1. Fragile State Index (gov_fsi)

4.   History of Conflict
>1. ?

In [0]:
# Python Libraries
from google.colab import auth
import uuid
import pandas as pd
import numpy as np
import io
import sys
import scipy
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

## Step 1: Authenticate Account

In [0]:
# Authenticate Google Cloud Storage
auth.authenticate_user()

In [0]:
# Project ID for WPSI bucket
project_id = 'wpsi-208318'

!gcloud config set project {project_id}

# WPSI bucket name
bucket_name = 'wpsi-june2018'


Updated property [core/project].


## Step 2: Read Model Input files into Colab

In [0]:
# Use the GCS Python API to read all the files in our bucket.
!pip install -q google-cloud-storage
from google.cloud import storage

gcs_client = storage.Client(project=project_id)
bucket = gcs_client.bucket(bucket_name)

In [0]:

# Create a list of all files in the bucket
files = list(bucket.list_blobs())

# Create a list of all model_input (mi) files
mi_files = []
for file in files:
  if "model_inputs" in str(file):
    mi_files.append(file)
    print(file)


<Blob: wpsi-june2018, pilot/model_inputs/>
<Blob: wpsi-june2018, pilot/model_inputs/deltares_SPI_month_anom_3m.csv>
<Blob: wpsi-june2018, pilot/model_inputs/hcss_eth_year_total.csv>
<Blob: wpsi-june2018, pilot/model_inputs/hcss_fsi_year_index.csv>
<Blob: wpsi-june2018, pilot/model_inputs/hcss_inf_year_total.csv>
<Blob: wpsi-june2018, pilot/model_inputs/ihe_ET_month_anom.csv>
<Blob: wpsi-june2018, pilot/model_inputs/ihe_ET_month_avg.csv>
<Blob: wpsi-june2018, pilot/model_inputs/wri_bws_chronic.csv>
<Blob: wpsi-june2018, pilot/model_inputs/wri_popden_year_avg.csv>
<Blob: wpsi-june2018, pilot/model_inputs/wri_popden_year_diff_1y.csv>
<Blob: wpsi-june2018, pilot/model_inputs/wri_popden_year_normdiff_1y.csv>


## Step 3: Define Functions

In [0]:
# Keep track of data attributes
earliest_years = []
latest_years = []

In [0]:
# Read files on GCS into Pandas Dataframe
def gcs_to_pandas(file_type):
  # Find the index of the file
  file_ind = [files.index(i) for i in files if file_type in str(i)]
  # Select the file
  selected_file = files[file_ind[0]]
  # Download data from file
  file_contents = selected_file.download_as_string()
  return file_contents


In [0]:
# Reformat data into Place, Month, Year format
def reformat_data(dataframe):
  df_melt = pd.melt(dataframe, id_vars='GID_2')
  df_melt["Month"] = df_melt["variable"].apply(lambda x: int(x[1:3]))
  df_melt["Year"] = df_melt["variable"].apply(lambda x: int(x[4:]))
  
  # Scrape details from dataset
  earliest_years.append(df_melt["Year"].min())
  latest_years.append(df_melt["Year"].max())
  
  # Clean up dataframe: drop unneeded fields and reset index
  df_melt.drop(["variable"], axis=1, inplace=True)
  df_melt.set_index(["GID_2", "Month", "Year"], inplace=True)
    
  return df_melt
  

In [0]:
def reformat_wri(file_type, field_name):
  selected_content = gcs_to_pandas(file_type)
#   df_pop = pd.read_csv(io.BytesIO(selected_content), header=0, index_col =0)
  df_pop = pd.read_csv(io.BytesIO(selected_content), header=0)
  df_pop.rename(columns={'value':field_name}, inplace=True)
  earliest_years.append(df_pop["Year"].min())
  latest_years.append(df_pop["Year"].max())
  
  return df_pop

In [0]:
# Runs all functions needed to create format-appropriate model input dataframe
def run_all(file_details):
  selected_content = gcs_to_pandas(file_details)
  df_sel = pd.read_csv(io.BytesIO(selected_content), header=0)
  dfm_sel = reformat_data(df_sel)
  return dfm_sel
  
  

In [0]:
# Merge datasets unto master dataframe
def merge_data(timespan, spatialspan, keep_dataframe, new_data):
  if timespan == "monthly" and spatialspan == "adm2":
    df_merge = pd.merge(keep_dataframe, new_data, how='left', left_index=True, right_index=True)
  elif timespan == "annual" and spatialspan == "adm0":
    df_merge = pd.merge(keep_dataframe, new_data, how='left', left_on=['Year', 'ISO'], right_on=['Year', 'ISO'])
  elif timespan == "annual" and spatialspan == "adm2":
    df_merge = pd.merge(keep_dataframe, new_data, how='left', left_on=['Year', 'GID_2'], right_on=['Year', 'GID_2'])
  return df_merge


## Step 4: Read in model inputs

### Deltares Data

In [0]:
# Standard Precipitation Index
precip_rename = {"SPI_month_anom_3m":'wat_precip_v1'}

df_spi_3m = run_all("SPI_month_anom_3m")
df_spi_3m.rename(columns={'value':'wat_precip_v1'}, inplace=True)
df_spi_3m.head(3)

,,,wat_precip_v1
GID_2,Month,Year,
AGO.1.1_1,1,1983,0.13
AGO.1.2_1,1,1983,0.20
AGO.1.3_1,1,1983,0.09


### IHE Data

In [0]:
# Evapotranspiration (ET)
et_rename = {"ET_month_anom":'wat_et_v1',
             "ET_month_avg":'wat_et_v2'}

df_et_anom = run_all("ET_month_anom")
df_et_anom.rename(columns={'value':'wat_et_v1'}, inplace=True)
df_et_avg = run_all("ET_month_avg")
df_et_avg.rename(columns={'value':'wat_et_v2'}, inplace=True)
df_et_avg.head(3)

,,,wat_et_v2
GID_2,Month,Year,
AGO.1.1_1,1,2003,110.00
AGO.1.2_1,1,2003,110.12
AGO.1.3_1,1,2003,88.60


### WRI Data

In [0]:
# Population Density
# No months -- will merge based on GID_2 and year

# popden_year_avg -> Population Density: average population density
# popden_year_diff -> Change in population density from year before
# popden_year_normdiff -> Percent difference in population density from year before (new-old)/old

wri_rename = {"popden_year_avg":'soc_pop_v1',
             "popden_year_diff":'soc_pop_v2',
             "popden_year_normdiff":'soc_pop_v3'}

df_pop_avg = reformat_wri("popden_year_avg", "soc_pop_v1")
df_pop_dif = reformat_wri("popden_year_diff", "soc_pop_v2")
df_pop_ndif = reformat_wri("popden_year_normdiff", "soc_pop_v3")
df_pop_ndif.head(3)

,GID_2,soc_pop_v3,Year
0,SHN.3.1_1,-0.014852,2001
1,SHN.2.1_1,-0.036166,2001
2,SHN.2.7_1,-0.035987,2001


In [0]:
# Baseline Water Stress (BWS)
# No dates attach -- will merge based on GID_2 only
selected_content = gcs_to_pandas("wri_bws")
df_bws = pd.read_csv(io.BytesIO(selected_content), header=0 )
df_bws["Year"] = 1997
df_bws.drop(['GID_0', "GID_1", "BWS_cat"], axis=1, inplace=True)
df_bws.rename(columns={'BWS_s':"wat_bws_v1"}, inplace=True)
df_temp = df_bws.copy()

for i in range(1998, 2017):
  df_temp['Year'] = i
  df_bws = df_bws.append(df_temp)

df_bws.head(3)

,GID_2,wat_bws_v1,Year
0,AGO.1.1_1,0.000000,1997
1,AGO.1.2_1,0.222302,1997
2,AGO.1.3_1,0.144228,1997


### HCSS Data

In [0]:
# All ACLED
acled_rename = {'fatalities': 'con_fat_v1',
               "scale": "con_scl_v1", "scale_lag": "con_scl_v2",
               "binary": "con_bin_v1", "binary_lag": "con_bin_v2",
               "scale_3ma": "con_scl_3m", "scale_6ma": "con_scl_6m", "scale_9ma": "con_scl_9m", "scale_12ma": "con_scl_12m", "scale_15ma": "con_scl_15m",
               "scale_18ma": "con_scl_18m", "scale_21ma": "con_scl_21m", "scale_24ma": "con_scl_24m", "scale_27ma": "con_scl_27m", "scale_30ma": "con_scl_30m",
               "scale_33ma": "con_scl_33m", "scale_36ma": "con_scl_36m", "scale_39ma": "con_scl_39m", "scale_42ma": "con_scl_42m", "scale_45ma": "con_scl_45m",
               "scale_48ma": "con_scl_48m", "scale_51ma": "con_scl_51m", "scale_54ma": "con_scl_54m", "scale_57ma": "con_scl_57m", "scale_60ma": "con_scl_60m",
               "binary_3ma": "con_bin_3m", "binary_6ma": "con_bin_6m", "binary_9ma": "con_bin_9m", "binary_12ma": "con_bin_12m", "binary_15ma": "con_bin_15m", 
               "binary_18ma": "con_bin_18m", "binary_21ma": "con_bin_21m", "binary_24ma": "con_bin_24m", "binary_27ma": "con_bin_27m", "binary_30ma": "con_bin_30m", 
               "binary_33ma": "con_bin_33m", "binary_36ma": "con_bin_36m", "binary_39ma": "con_bin_39m", "binary_42ma": "con_bin_42m", "binary_45ma": "con_bin_45m", 
               "binary_48ma": "con_bin_48m", "binary_51ma": "con_bin_51m", "binary_54ma": "con_bin_54m", "binary_57ma": "con_bin_57m", "binary_60ma": "con_bin_60m"}

selected_content = gcs_to_pandas("acled_wri_amd2.txt")
df_acled = pd.read_csv(io.BytesIO(selected_content), header=0,sep='\t' )
df_acled["Month"] = df_acled["month"].apply(lambda x: int(x.split('-')[1]))
df_acled["Year"] = df_acled["month"].apply(lambda x: int(x.split('-')[0]))
df_acled.set_index(["GID_2", "Month", "Year"], inplace=True)
df_acled.drop(["country", "month"], axis=1, inplace = True)
df_acled.rename(columns=acled_rename, inplace=True)
df_acled.head(10)

iso3c  con_fat_v1  con_scl_v1  con_bin_v1  con_scl_v2  \
GID_2       Month Year                                                         
DZA.10.10_1 2     1997   DZA         0.0    0.000000           0    0.000000   
            3     1997   DZA         0.0    0.000000           0    0.000000   
            4     1997   DZA        47.0    1.672098           1    0.000000   
            5     1997   DZA        29.0    1.462398           1    1.672098   
            6     1997   DZA         0.0    0.000000           0    1.462398   
            7     1997   DZA         0.0    0.000000           0    0.000000   
            8     1997   DZA         0.0    0.000000           0    0.000000   
            9     1997   DZA        47.0    1.672098           1    0.000000   
            10    1997   DZA         0.0    0.000000           0    1.672098   
            11    1997   DZA         0.0    0.000000           0    0.000000   

                        con_bin_v2  con_scl_3m  con_scl_6m  con_scl_9m  \
GID_2       Month Year                                                   
DZA.10.10_1 2     1997           0         NaN         NaN         NaN   
            3     1997           0         NaN         NaN         NaN   
            4     1997           0    0.000000         NaN         NaN   
            5     1997           1    0.557366         NaN         NaN   
            6     1997           1    1.044832         NaN         NaN   
            7     1997           0    1.044832    0.522416         NaN   
            8     1997           0    0.487466    0.522416         NaN   
            9     1997           0    0.000000    0.522416         NaN   
            10    1997           1    0.557366    0.801099    0.534066   
            11    1997           0    0.557366    0.522416    0.534066   

                        con_scl_12m     ...       con_bin_33m  con_bin_36m  \
GID_2       Month Year                  ...                                  
DZA.10.10_1 2     1997          NaN     ...               NaN          NaN   
            3     1997          NaN     ...               NaN          NaN   
            4     1997          NaN     ...               NaN          NaN   
            5     1997          NaN     ...               NaN          NaN   
            6     1997          NaN     ...               NaN          NaN   
            7     1997          NaN     ...               NaN          NaN   
            8     1997          NaN     ...               NaN          NaN   
            9     1997          NaN     ...               NaN          NaN   
            10    1997          NaN     ...               NaN          NaN   
            11    1997          NaN     ...               NaN          NaN   

                        con_bin_39m  con_bin_42m  con_bin_45m  con_bin_48m  \
GID_2       Month Year                                                       
DZA.10.10_1 2     1997          NaN          NaN          NaN          NaN   
            3     1997          NaN          NaN          NaN          NaN   
            4     1997          NaN          NaN          NaN          NaN   
            5     1997          NaN          NaN          NaN          NaN   
            6     1997          NaN          NaN          NaN          NaN   
            7     1997          NaN          NaN          NaN          NaN   
            8     1997          NaN          NaN          NaN          NaN   
            9     1997          NaN          NaN          NaN          NaN   
            10    1997          NaN          NaN          NaN          NaN   
            11    1997          NaN          NaN          NaN          NaN   

                        con_bin_51m  con_bin_54m  con_bin_57m  con_bin_60m  
GID_2       Month Year                                                      
DZA.10.10_1 2     1997          NaN          NaN          NaN          NaN  
            3     1997          NaN          NaN          NaN       

In [0]:
# Infant Mortality (INF)
# No months -- will merge based on GID_2, year, and ISO
# SH.DTH.IMRT -> Number of infant deaths
# SP.DYN.IMRT.FE.IN -> Mortality rate (infant, female) (per 1000 births)
# SP.DYN.IMRT.IN -> Mortality rate (infant) (per 1000 births)
# SP.DYN.IMRT.MA.IN -> Mortality rate (infant, male) (per 1000 births)
# wdi_mortinf -> Mortality rate (infant) (per 1000 births)
# wdi_mortinff -> Mortality rate (infant, female) (per 1000 births)
# wdi_mortinfm -> Mortality rate (infant, male) (per 1000 births)
inf_rename = {"SH.DTH.IMRT":'soc_inf_v1',
             "SP.DYN.IMRT.FE.IN":'soc_inf_v2',
             "SP.DYN.IMRT.IN":'soc_inf_v3',
             "SP.DYN.IMRT.MA.IN":'soc_inf_v4',
             "wdi_mortinf":'soc_inf_v5',
             "wdi_mortinff":'soc_inf_v6',
             "wdi_mortinfm":'soc_inf_v7'}
# Read in data
selected_content = gcs_to_pandas("hcss_inf_year")
df_inf = pd.read_csv(io.BytesIO(selected_content), header=0 )
df_inf.rename(columns={'year':"Year", "iso3c":'ISO'}, inplace=True)
df_inf.rename(columns=inf_rename, inplace=True)

df_inf.head(3)

,ISO,Year,soc_inf_v1,soc_inf_v2,soc_inf_v3,soc_inf_v4,soc_inf_v5,soc_inf_v6,soc_inf_v7
0,DZA,1997,23730.0,NaN,35.4,NaN,35.4,NaN,NaN
1,DZA,1998,22465.0,NaN,34.8,NaN,34.8,NaN,NaN
2,DZA,1999,21508.0,NaN,34.4,NaN,34.4,NaN,NaN


In [0]:
# Ethnic/Religious Fractionalism (ETH)
# No months -- will merge based on GID_2, year, and ISO
# al_ethnic -> Ethnic Fractionalization
# al_language -> Language Fractionalization
# al_religion -> Religion Fractionalization
# cspv_ethviol -> Magnitude score of episode(s) of ethnic violence
# cspv_ethwar -> Magnitude score of episode(s) of ethnic warfare
# iaep_ebbp -> Ethnicity Based on Banning of Parties

eth_rename = {"al_ethnic":'soc_eth_v1',
             "al_language":'soc_eth_v2',
             "al_religion":'soc_eth_v3',
             "cspv_ethviol":'soc_eth_v4',
             "cspv_ethwar":'soc_eth_v5',
             "iaep_ebbp":'soc_eth_v6'}
# Read in data
selected_content = gcs_to_pandas("hcss_eth_year")
df_eth = pd.read_csv(io.BytesIO(selected_content), header=0 )
df_eth.rename(columns={'year':"Year", "iso3c":'ISO'}, inplace=True)
df_eth.rename(columns=eth_rename, inplace=True)

df_eth.head(3)

,ISO,Year,soc_eth_v1,soc_eth_v2,soc_eth_v3,soc_eth_v4,soc_eth_v5,soc_eth_v6
0,DZA,1997,0.3394,0.442662,0.009128,0.0,0.0,0.0
1,DZA,1998,0.3394,0.442662,0.009128,0.0,0.0,0.0
2,DZA,1999,0.3394,0.442662,0.009128,0.0,0.0,0.0


In [0]:
# Fragile State Index (FSI)
# No months -- will merge based on GID_2, year, and ISO
# ffp_fsi -> Frafile State Index

fsi_rename = {"ffp_fsi":'gov_fsi_v1'}
# Read in data
selected_content = gcs_to_pandas("hcss_fsi_year")
df_fsi = pd.read_csv(io.BytesIO(selected_content), header=0 )
df_fsi.rename(columns={'year':"Year", "iso3c":'ISO'}, inplace=True)
df_fsi.rename(columns=fsi_rename, inplace=True)

df_fsi.head(3)

,ISO,Year,gov_fsi_v1
0,DZA,1997,NaN
1,DZA,1998,NaN
2,DZA,1999,NaN


## Step 4: Create master dataframe

In [0]:
selected_content = gcs_to_pandas("africa_gadm2")
df_adm = pd.read_csv(io.BytesIO(selected_content), header=0)
all_adm = df_adm["GID_2"].tolist()

# Define start and end years for model
year_start = 1997
year_end = 2017
print ("From", str(year_start), "to", str(year_end))
# Calculate the number of unique combinataions of Adm units and months
ua = len(all_adm)
um = (year_end+1 - year_start)*12
uc = um * ua

print ("Number of Adminstrative Units:", str(ua))
print ("Number of months:", str(um))
print ("Number of unique combinations of Adm Units and months:", str(uc))

From 1997 to 2017
Number of Adminstrative Units: 6395
Number of months: 252
Number of unique combinations of Adm Units and months: 1611540


In [0]:
# Create master dataframe
df = pd.DataFrame(index=range(0,uc))
# Add in Adminstrative Units
df["GID_2"] = all_adm*um
# Add in Months
all_months = [[i] * ua for i in range(1,13)]
flattened_list = [y for x in all_months for y in x]*(year_end+1-year_start)
df["Month"] = flattened_list
# Add in Year
all_years = [[i] * ua*12 for i in range(year_start,year_end+1)]
flattened_list2 = [y for x in all_years for y in x]
df["Year"] = flattened_list2
# Add ISO code
df["ISO"] = df['GID_2'].apply(lambda x: x[0:3])
df_train = df.copy()
df.head(3)

,GID_2,Month,Year,ISO
0,AGO.1.1_1,1,1997,AGO
1,AGO.1.2_1,1,1997,AGO
2,AGO.1.3_1,1,1997,AGO


## Step 6a: Merge inputs into master dataframe (annual data)

In [0]:
# Add in annual data
#merge_data (temporal resolution, spatial resolution, master dataframe, new data, field )
# HCSS
df_all = merge_data("annual", "adm0", df, df_inf) #soc_inf
df_all = merge_data("annual","adm0", df_all, df_eth) #soc_eth
df_all = merge_data("annual", "adm0",df_all, df_fsi) #gov_fsi
df_all = merge_data("annual", "adm0", df_all, df_fsi) #gov_fsi
# WRI
df_all = merge_data("annual", "adm2", df_all, df_pop_avg) #soc_pop
df_all = merge_data("annual", "adm2", df_all, df_pop_dif) #soc_pop
df_all = merge_data("annual", "adm2", df_all, df_pop_ndif) #soc_pop
df_all = merge_data("annual", "adm2", df_all, df_bws) #soc_bws


df_all.head(3)

,GID_2,Month,Year,ISO,soc_inf_v1,soc_inf_v2,soc_inf_v3,soc_inf_v4,soc_inf_v5,soc_inf_v6,...,soc_eth_v3,soc_eth_v4,soc_eth_v5,soc_eth_v6,gov_fsi_v1_x,gov_fsi_v1_y,soc_pop_v1,soc_pop_v2,soc_pop_v3,wat_bws_v1
0,AGO.1.1_1,1,1997,AGO,94936.0,NaN,129.9,NaN,129.9,NaN,...,0.627644,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.000000
1,AGO.1.2_1,1,1997,AGO,94936.0,NaN,129.9,NaN,129.9,NaN,...,0.627644,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.222302
2,AGO.1.3_1,1,1997,AGO,94936.0,NaN,129.9,NaN,129.9,NaN,...,0.627644,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.144228


## Step 6b: Merge inputs into master dataframe (monthly)

In [0]:
# Reset index
df_all.set_index(["GID_2", "Month", "Year"], inplace=True)
df_all.head(3)

,,,ISO,soc_inf_v1,soc_inf_v2,soc_inf_v3,soc_inf_v4,soc_inf_v5,soc_inf_v6,soc_inf_v7,soc_eth_v1,soc_eth_v2,soc_eth_v3,soc_eth_v4,soc_eth_v5,soc_eth_v6,gov_fsi_v1_x,gov_fsi_v1_y,soc_pop_v1,soc_pop_v2,soc_pop_v3,wat_bws_v1
GID_2,Month,Year,,,,,,,,,,,,,,,,,,,,
AGO.1.1_1,1,1997,AGO,94936.0,NaN,129.9,NaN,129.9,NaN,NaN,0.78672,0.787019,0.627644,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.000000
AGO.1.2_1,1,1997,AGO,94936.0,NaN,129.9,NaN,129.9,NaN,NaN,0.78672,0.787019,0.627644,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.222302
AGO.1.3_1,1,1997,AGO,94936.0,NaN,129.9,NaN,129.9,NaN,NaN,0.78672,0.787019,0.627644,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.144228


In [0]:
# Add in monthly data
#merge_data (temporal resolution, spatial resolution, master dataframe, new data, field )
# IHE
df_all = merge_data("monthly", "adm2", df_all, df_et_anom) 
df_all = merge_data("monthly", "adm2", df_all, df_et_avg)
# Deltares
df_all = merge_data("monthly", "adm2", df_all, df_spi_3m) 
# HCSS
df_all = merge_data("monthly", "adm2", df_all, df_acled)
df_all.head(30)

NameError: ignored